In [7]:
from __future__ import division
from __future__ import print_function
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# %tensorflow_version 1.14
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy.linalg as la
import scipy.io as sio
import math
import sys
import time
import pdb
import matplotlib.pyplot as plt
import keras.backend as K
import cvxpy as cp
import blocksparsetoolbox as bst

# Problem

In [ ]:

class Generator(object):
    def __init__(self, A, **kwargs):
        self.A = A
        M, N = A.shape
        vars(self).update(kwargs)
        self.x_ = tf.placeholder(tf.float32, (N, None), name='x')
        self.y_ = tf.placeholder(tf.float32, (M, None), name='y')


class TFGenerator(Generator):
    def __init__(self, **kwargs):
        Generator.__init__(self, **kwargs)

    def __call__(self, sess):
        "generates y,x pair for training"
        return sess.run((self.ygen_, self.xgen_))


def block_gaussian_trial(m=128, L=32, B=16, MC=1000, pnz=.1, SNR_dB=20):
    N = B * L  # N is the length of a the unknown block-sparse x
    A2 = np.random.normal(size=(m, N), scale=1.0 / math.sqrt(m)).astype(np.float32)
    # pdb.set_trace()
    A2 = sio.loadmat('small_normalized_D.mat')
    A2 = A2.get('D')
    A=A2.astype('float32')
    A_ = tf.constant(A, name='A')
    prob = TFGenerator(A=A, A_=A_, kappa=None, SNR=SNR_dB)
    #pdb.set_trace()
    # prob.A=A
    prob.name = 'block sparse, Gaussian A'
    prob.L = L
    prob.B = B
    prob.N = N
    prob.m = m
    # prob.SNR_dB = SNR_dB
    prob.pnz = pnz

    # Create tf vectors
    active_blocks_ = tf.to_float(tf.random_uniform((L, 1 , MC)) < pnz)
    ones_ = tf.ones([L, B, MC])

    product_ = tf.multiply(active_blocks_, ones_)
    xgen_ = tf.reshape(product_, [L * B, MC])
    xgen_ = tf.multiply(xgen_, tf.random_normal((N, MC), 0, 1))

    # you should probably change the way noise_var is calculated
    noise_var = pnz * N / m * math.pow(10., -SNR_dB / 10.)
    ygen_ = tf.matmul(A_, xgen_) + tf.random_normal((m, MC), stddev=math.sqrt(noise_var))
    #ygen_ = tf.matmul(A_, xgen_)

    active_blocks_val = (np.random.uniform(0, 1, (L, MC)) < pnz).astype(np.float32)
    active_entries_val = np.repeat(active_blocks_val, B, axis=0)
    xval = np.multiply(active_entries_val, np.random.normal(0, 1, (N, MC)))
    yval = np.matmul(A, xval) + np.random.normal(0, math.sqrt(noise_var), (m, MC))
    #yval = np.matmul(A,xval)

    prob.xgen_ = xgen_
    prob.ygen_ = ygen_
    prob.xval = xval
    prob.yval = yval
    prob.noise_var = noise_var
    prob.noise = np.random.normal(0, math.sqrt(noise_var), (m))

    return prob

def createsparse(N,n):
    sig = np.zeros(N)
    k = np.random.randint(0, N, n)
    for a in range(n):
        for j in range(N):
            if j == k[a]:
                sig[j] = 1
    return sig

def new_block_gaussian_trial(m=128, L=32, B=16, MC=1000, sparsity=2, SNR_dB=20):
    N = B * L  # N is the length of a the unknown block-sparse x
    pnz = 0.1
    # A2 = np.random.normal(size=(m, N), scale=1.0 / math.sqrt(m)).astype(np.float32)
    A2 = sio.loadmat('Dblocksparse.mat')
    A2 = A2.get('D')
    A=A2
    A_ = tf.constant(A, name='A')
    prob = TFGenerator(A=A, A_=A_, kappa=None, SNR=SNR_dB)
    #pdb.set_trace()
    # prob.A=A
    prob.name = 'block sparse, Gaussian A'
    prob.L = L
    prob.B = B
    prob.N = N
    # prob.SNR_dB = SNR_dB
    prob.pnz = pnz

    # Create tf vectors

    o = np.zeros((sparsity, sparsity))
    o[:, 0] = np.random.randint(0, L, sparsity)
    active_blocks_ = tf.to_float(tf.random_uniform((L, 1 , MC)) < pnz)
    ones_ = tf.ones([L, B, MC])

    product_ = tf.multiply(active_blocks_, ones_)
    xgen_ = tf.reshape(product_, [L * B, MC])
    xgen_ = tf.multiply(xgen_, tf.random_normal((N,MC), 0, 1 ))

    # you should probably change the way noise_var is calculated
    noise_var = pnz * N / m * math.pow(10., -SNR_dB / 10.)
    ygen_ = tf.matmul(A_, xgen_) + tf.random_normal((m, MC), stddev=math.sqrt(noise_var))
    #ygen_ = tf.matmul(A_, xgen_)

    active_blocks_val = np.zeros((L,MC))
    for i in range(MC):
        active_blocks_val[:,i] = createsparse(L, sparsity)
    #pdb.set_trace()
    active_entries_val = np.repeat(active_blocks_val, B, axis=0)
    xval = np.multiply(active_entries_val, np.random.normal(0, 1, (N, MC)))
    yval = np.matmul(A, xval) + np.random.normal(0, math.sqrt(noise_var), (m, MC))
    #yval = np.matmul(A,xval)

    prob.xgen_ = xgen_
    prob.ygen_ = ygen_
    prob.xval = xval
    prob.yval = yval
    prob.noise_var = noise_var
    prob.noise = np.random.normal(0, math.sqrt(noise_var), (m, MC))

    return prob

# Network

In [ ]:
import cvxpy as cp

def R(D, n, d): #function to compute the generalized coherence for D = B^TD
    n_y, n_x = D.shape
    R = np.zeros((n, n))
    for k in range(0, n):
        for l in range(k, n):
            I = np.zeros((n_x, n_x))
            I_s = np.zeros((n, n))
            I[l * d:(l + 1) * d, k * d:(k + 1) * d] = np.ones(d)
            I_s[l, k] = 1
            if k == l:
                R = R + cp.norm(cp.multiply(I, D), 2) * I_s
            else:
                R = R + cp.norm(cp.multiply(I, D), 2) * I_s + cp.norm(cp.multiply(I.T, D), 2) * I_s.T

    return R

def simple_soft_threshold(r_, lam_):
    "implement a soft threshold function y=sign(r)*max(0,abs(r)-lam)"
    lam_ = tf.maximum(lam_, 0)
    #pdb.set_trace()
    return tf.sign(r_) * tf.maximum(tf.abs(r_) - lam_, 0)

def block_soft_threshold(X_, al_):
    B = 15
    L = 5
    shape = K.shape(X_)
    pool_shape1 = tf.stack([B, L, shape[1]])
    al_ = tf.maximum(al_, 0)
    Xnew_ = K.reshape(X_, pool_shape1)  # i'th coloumn of X_
    r_ = tf.sqrt(tf.reduce_sum((Xnew_ ** 2), 1))
    r_ = tf.maximum(.0, 1 - tf.math.divide_no_nan(al_ , r_))
    shaper = K.shape(r_)
    pool_r = tf.stack([B, 1, shaper[1]])
    r_ = K.reshape(r_, pool_r)  # Diesen reshape Befehl auch zu K.reshape ändern
    pool_shape2 = tf.stack([B * L, shape[1]])

    Xnew_ = tf.multiply(Xnew_, r_)

    return K.reshape(Xnew_, pool_shape2)  # transpose, bc of the list and tf.stack() operator

def block_soft_threshold_elastic_net(X_, al_, la_):
    B = 64
    L = 1

    shape = K.shape(X_)
    pool_shape1 = tf.stack([B, L, shape[1]])
    al_ = tf.maximum(al_, 0)
    Xnew_ = K.reshape(X_, pool_shape1)  # i'th coloumn of X_
    r_ = tf.sqrt(tf.reduce_sum((Xnew_ ** 2), 1))
    r_ = tf.maximum(.0, 1 - tf.math.divide_no_nan(al_ , r_))
    shaper = K.shape(r_)
    pool_r = tf.stack([B, 1, shaper[1]])
    r_ = K.reshape(r_, pool_r)  # Diesen reshape Befehl auch zu K.reshape ändern
    pool_shape2 = tf.stack([B * L, shape[1]])

    Xnew_ = tf.multiply(Xnew_, r_)*(1+la_)**(-1)

    return K.reshape(Xnew_, pool_shape2)  # transpose, bc of the list and tf.stack() operator

def build_LBISTA(prob,T,initial_lambda=.1,untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft=block_soft_threshold
    layers = []
    A = prob.A
    M,N = A.shape
    B = A.T / (1.01 * la.norm(A,2)**2)
    B_ =  tf.Variable(B,dtype=tf.float32,name='B_0')
    By_ = tf.matmul(B_,prob.y_)
    S_ = tf.Variable( np.identity(N) - np.matmul(B,A),dtype=tf.float32,name='S_0')
    #pdb.set_trace()
    #layers.append( ('Linear',By_,None) )
    
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    #if getattr(prob,'iid',True) == False:
    #    # create a parameter for each coordinate in x
    #    initial_lambda = initial_lambda*np.ones( (N,1),dtype=np.float32 )
    lam0_ = tf.Variable( initial_lambda,name='lam_0')
    xhat_ = blocksoft( By_, lam0_)
    layers.append( ('LBISTA T=1',xhat_, (lam0_,B_, S_) ) )
    #pdb.set_trace()
    for t in range(1,T):
        lam_ = tf.Variable( initial_lambda,name='lam_{0}'.format(t) )
        xhat_ = blocksoft( tf.matmul(S_,xhat_) + tf.matmul(B_,prob.y_), lam_ )
        layers.append( ('LBISTA T='+str(t+1),xhat_,(lam_,B_, S_)) )

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers


def build_LBISTA_untied(prob,T,initial_lambda=.1,untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm

     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft=block_soft_threshold
    layers = []
    A = prob.A
    M,N = A.shape
    B = A.T / (1.01 * la.norm(A,2)**2)
    B_ =  tf.Variable(B,dtype=tf.float32,name='B_0')
    S_ = tf.Variable( np.identity(N) - np.matmul(B,A),dtype=tf.float32,name='S_0')
    By_ = tf.matmul(B_,prob.y_)
    #pdb.set_trace()
    #layers.append( ('Linear',By_,None) )
    
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    #if getattr(prob,'iid',True) == False:
    #    # create a parameter for each coordinate in x
    #    initial_lambda = initial_lambda*np.ones( (N,1),dtype=np.float32 )
    lam0_ = tf.Variable( initial_lambda,name='lam_0')
    xhat_ = blocksoft( By_, lam0_)
    layers.append( ('LBISTA T=1',xhat_, (lam0_,B_, S_) ) )
    #pdb.set_trace()
    for t in range(1,T):
        B_ =  tf.Variable(B,dtype=tf.float32,name='B_{0}'.format(t) )
        S_ = tf.Variable( np.identity(N) - np.matmul(B,A),dtype=tf.float32,name='S_{0}'.format(t) )
        lam_ = tf.Variable( initial_lambda,name='lam_{0}'.format(t) )
        xhat_ = blocksoft( tf.matmul(S_,xhat_) + tf.matmul(B_,prob.y_), lam_ )
        layers.append( ('LBISTA T='+str(t+1),xhat_,(lam_,B_, S_)) )

    """

    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers

def build_ALISTA(prob, T, initial_lambda=.1, initial_gamma=1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = simple_soft_threshold
    layers = []
    A = prob.A
    M, N = A.shape
    Wmat = sio.loadmat('W.mat') #W has to be precomputed before running the script
    W = Wmat.get('W')
    gamma = initial_gamma
    gamma0_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_0')
    W_ = tf.Variable(np.transpose(W), dtype=tf.float32, trainable = False)
    layers.append(('Linear', tf.matmul(gamma0_*W_, prob.y_), None))
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(tf.matmul(gamma0_*W_, prob.y_), lam0_)
    layers.append(('LBISTA T=1', xhat_, (lam0_,gamma0_)))
    # pdb.set_trace()
    for t in range(1, T):
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        gamma_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_{0}'.format(t))
        xhat_ = blocksoft(xhat_ - tf.matmul(gamma_*W_, tf.matmul(prob.A_, xhat_) - prob.y_), lam_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_,gamma_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers

def build_TiBLISTA(prob, T, initial_lambda=.1, initial_gamma=1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = block_soft_threshold
    layers = []
    A = prob.A
    M, N = A.shape
    #Wmat = sio.loadmat('W.mat') #W has to be precomputed before running the script
    W = A
    gamma = initial_gamma
    gamma0_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_0')
    W_ = tf.Variable(np.transpose(W), dtype=tf.float32, trainable = True)
    #layers.append(('Linear', tf.matmul(gamma0_*W_, prob.y_), None))
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(tf.matmul(gamma0_*W_, prob.y_), lam0_)
    layers.append(('LBISTA T=1', xhat_, (lam0_,gamma0_,W_)))
    # pdb.set_trace()
    for t in range(1, T):
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        gamma_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_{0}'.format(t))
        xhat_ = blocksoft(xhat_ - tf.matmul(gamma_*W_, tf.matmul(prob.A_, xhat_) - prob.y_), lam_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_,gamma_,W_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers


def build_LBISTA_CPSS(prob, T, initial_lambda=.1, initial_gamma=1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = block_soft_threshold
    layers = []

    A = prob.A
    M, N = A.shape
    #Wmat = sio.loadmat('W.mat') #W has to be precomputed before running the script
    W = A
    gamma = initial_gamma
    gamma0_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_0')
    W_ = tf.Variable(np.transpose(W), dtype=tf.float32, trainable = True)
    #layers.append(('Linear', tf.matmul(gamma0_*W_, prob.y_), None))
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(tf.matmul(gamma0_*W_, prob.y_), lam0_)
    layers.append(('LBISTA T=1', xhat_, (lam0_,gamma0_,W_)))
    # pdb.set_trace()
    for t in range(1, T):
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        W_ = tf.Variable(np.transpose(W), dtype=tf.float32, name='W_{0}'.format(t), trainable = True)
        gamma_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_{0}'.format(t))
        xhat_ = blocksoft(xhat_ - tf.matmul(gamma_*W_, tf.matmul(prob.A_, xhat_) - prob.y_), lam_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_,gamma_,W_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers

def build_BALISTA(prob, T, initial_lambda=.1, initial_gamma=1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = block_soft_threshold
    layers = []
    A = prob.A
    M, N = A.shape
    Wmat = sio.loadmat('Wblock.mat') #W has to be precomputed before running the script
    W = Wmat.get('W')
    gamma = initial_gamma
    gamma0_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_0')
    W_ = tf.Variable(np.transpose(W), dtype=tf.float32, trainable = False)
    #layers.append(('Linear', tf.matmul(gamma0_*W_, prob.y_), None))
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(tf.matmul(gamma0_*W_, prob.y_), lam0_)
    layers.append(('LBISTA T=1', xhat_, (lam0_,gamma0_)))
    # pdb.set_trace()
    for t in range(1, T):
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        gamma_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_{0}'.format(t))
        xhat_ = blocksoft(xhat_ - tf.matmul(gamma_*W_, tf.matmul(prob.A_, xhat_) - prob.y_), lam_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_,gamma_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers, W


def build_BALISTA_v2(prob, T, initial_lambda=.1, initial_gamma=1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = block_soft_threshold
    layers = []
    A = prob.A
    gamma = initial_gamma
    gamma0_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_0')

    W = bst.linalg.pinv(A.T)

    W_ = tf.Variable(W.T, dtype=tf.float32, trainable = False)
    #layers.append(('Linear', tf.matmul(gamma0_*W_, prob.y_), None))
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(tf.matmul(gamma0_*W_, prob.y_), lam0_)
    layers.append(('LBISTA T=1', xhat_, (lam0_, gamma0_)))
    # pdb.set_trace()
    for t in range(1, T):
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        gamma_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_{0}'.format(t))
        xhat_ = blocksoft(xhat_ - tf.matmul(gamma_*W_, tf.matmul(prob.A_, xhat_) - prob.y_), lam_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_, gamma_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers, W

def build_BALISTA_v3(prob, T, initial_lambda=.1, initial_gamma=1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = block_soft_threshold
    layers = []
    A = prob.A
    M, N = A.shape
    W = A
    gamma = initial_gamma
    gamma0_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_0')

    W = bst.compute_W_v1(A, 32, 16)

    pdb.set_trace()

    W_ = tf.Variable(W.T, dtype=tf.float32, trainable = False)
    #layers.append(('Linear', tf.matmul(gamma0_*W_, prob.y_), None))
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(tf.matmul(gamma0_*W_, prob.y_), lam0_)
    layers.append(('LBISTA T=1', xhat_, (lam0_, gamma0_)))
    # pdb.set_trace()
    for t in range(1, T):
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        gamma_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_{0}'.format(t))
        xhat_ = blocksoft(xhat_ - tf.matmul(gamma_*W_, tf.matmul(prob.A_, xhat_) - prob.y_), lam_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_,gamma_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers, W

def build_BALISTA_v4(prob, T, initial_lambda=.1, initial_gamma=1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = block_soft_threshold
    layers = []
    A = prob.A
    gamma = initial_gamma
    gamma0_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_0')
    # pdb.set_trace()
    #W = bst.compute_W_Lagrange(A, 32, 3)

    #solve the following problem to get the analytical weight matrix
    # D = prob.A
    # n = prob.L
    # d = prob.B
    # m = prob.m
    # B_up = cp.Variable((m, n * d))
    #
    # I = np.kron(np.eye(n), np.ones((d, d)))
    # k = np.tile(np.eye(d), (1, n)).T
    # b = cp.multiply(B_up.T @ D, I)
    # b = cp.matmul(b, k)  # extracting the diagonal blocks of D^TB
    # constraints = [b == k]
    # objective = cp.Minimize(1 / d * (cp.norm(B_up.T @ D, 'fro')))
    # prob = cp.Problem(objective, constraints)
    # result = prob.solve()
    #
    # W = B_up.value
    Wmat = sio.loadmat('sparse_case_W_up.mat')
    W = Wmat.get('W')
    W_ = tf.Variable(W.T, dtype=tf.float32, trainable = False)
    #layers.append(('Linear', tf.matmul(gamma0_*W_, prob.y_), None))
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(tf.matmul(tf.math.abs(gamma0_)*W_, prob.y_), lam0_)
    layers.append(('LBISTA T=1', xhat_, (lam0_, gamma0_)))
    # pdb.set_trace()
    for t in range(1, T):
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        gamma_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_{0}'.format(t))
        xhat_ = blocksoft(xhat_ - tf.matmul(tf.math.abs(gamma_)*W_, tf.matmul(prob.A_, xhat_) - prob.y_), lam_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_,gamma_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers, W

def build_BALISTA_v5(prob, T, initial_lambda=.1, initial_gamma=1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = block_soft_threshold
    layers = []
    A = prob.A
    gamma = initial_gamma
    gamma0_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_0')
    # pdb.set_trace()
    #W = bst.compute_W_Lagrange(A, 32, 3)
    # solve the following problem to get the analytical weight matrix
    # D = prob.A
    # n = prob.L
    # d = prob.B
    # m = prob.m
    # B_cvx = cp.Variable((m, n * d))
    #
    # I = np.kron(np.eye(n), np.ones((d, d)))
    # k = np.tile(np.eye(d), (1, n)).T
    # b = cp.multiply(B_cvx.T @ D, I)
    # b = cp.matmul(b, k)  # extracting the diagonal blocks of D^TB
    # constraints = [b == k]
    # objective = cp.Minimize(1/d*cp.max(R(B_cvx.T@D-np.eye(n*d), n, d)))
    # prob = cp.Problem(objective, constraints)
    # result = prob.solve()
    # W = B_cvx.value
    Wmat = sio.loadmat('small_normalized_W_cvx.mat')
    W = Wmat.get('W_cvx')
    W_ = tf.Variable(W.T, dtype=tf.float32, trainable = False)
    #layers.append(('Linear', tf.matmul(gamma0_*W_, prob.y_), None))
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(tf.matmul(tf.math.abs(gamma0_)*W_, prob.y_), lam0_)
    layers.append(('LBISTA T=1', xhat_, (lam0_, gamma0_)))
    # pdb.set_trace()
    for t in range(1, T):
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        gamma_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_{0}'.format(t))
        xhat_ = blocksoft(xhat_ - tf.matmul(tf.math.abs(gamma_)*W_, tf.matmul(prob.A_, xhat_) - prob.y_), lam_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_,gamma_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers, W

def build_TiLISTA(prob, T, initial_lambda=.1, initial_gamma=1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = simple_soft_threshold
    layers = []
    A = prob.A
    M, N = A.shape
    #Wmat = sio.loadmat('W.mat') #W has to be precomputed before running the script
    W = A
    gamma = initial_gamma
    gamma0_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_0')
    W_ = tf.Variable(np.transpose(W), dtype=tf.float32, trainable = True)
    layers.append(('Linear', tf.matmul(2*gamma0_*W_, prob.y_), None))
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(tf.matmul(2*gamma0_*W_, prob.y_), lam0_)
    layers.append(('LBISTA T=1', xhat_, (lam0_,gamma0_)))
    # pdb.set_trace()
    for t in range(1, T):
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        gamma_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_{0}'.format(t))
        xhat_ = blocksoft(xhat_ - tf.matmul(2*gamma_*W_, tf.matmul(prob.A_, xhat_) - prob.y_), lam_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_,gamma_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers


def build_TiBLISTA(prob, T, initial_lambda=.1, initial_gamma=1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = block_soft_threshold
    layers = []
    A = prob.A
    M, N = A.shape
    #Wmat = sio.loadmat('W.mat') #W has to be precomputed before running the script
    W = A
    gamma = initial_gamma
    gamma0_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_0')
    W_ = tf.Variable(np.transpose(W), dtype=tf.float32, trainable = True)
    layers.append(('Linear', tf.matmul(gamma0_*W_, prob.y_), None))
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(tf.matmul(gamma0_*W_, prob.y_), lam0_)
    layers.append(('LBISTA T=1', xhat_, (lam0_,gamma0_)))
    # pdb.set_trace()
    for t in range(1, T):
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        gamma_ = tf.Variable(gamma, dtype=tf.float32, name='gamma_{0}'.format(t))
        xhat_ = blocksoft(xhat_ - tf.matmul(gamma_*W_, tf.matmul(prob.A_, xhat_) - prob.y_), lam_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_,gamma_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers

def build_LBFISTA(prob,T,initial_lambda=.1,untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft=block_soft_threshold
    layers = []
    A = prob.A
    M,N = A.shape
    B = A.T / (1.01 * la.norm(A,2)**2)
    B_ =  tf.Variable(B,dtype=tf.float32,name='B_0')
    By_ = tf.matmul(B_,prob.y_)
    S_ = tf.Variable( np.identity(N) - np.matmul(B,A),dtype=tf.float32,name='S_0')
    #pdb.set_trace()
    layers.append( ('Linear',By_,None) )
    
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    #if getattr(prob,'iid',True) == False:
    #    # create a parameter for each coordinate in x
    #    initial_lambda = initial_lambda*np.ones( (N,1),dtype=np.float32 )
    lam0_ = tf.Variable( initial_lambda,name='lam_0')
    xhat_ = blocksoft( By_, lam0_)
    tk = (1+np.sqrt(1+4*1**2))*2**(-1)
    z_ = xhat_
    layers.append( ('LBISTA T=1',xhat_, (lam0_,) ) )
    #pdb.set_trace()
    for t in range(1,T):
        t_prev = tk
        xhat_prev_ = xhat_ 
        lam_ = tf.Variable( initial_lambda,name='lam_{0}'.format(t) )
        xhat_ = blocksoft( tf.matmul(S_,z_) + By_, lam_ )
        tk = (1+np.sqrt(1+4*t_prev**2))*2**(-1)
        z_ = xhat_ + (t_prev-1)*(tk)**(-1)*(xhat_-xhat_prev_)
        layers.append( ('LBISTA T='+str(t+1),xhat_,(lam_,)) )

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers


def build_LBFISTA_idea(prob, T, initial_lambda=.1, untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    blocksoft = block_soft_threshold
    layers = []
    A = prob.A
    M, N = A.shape
    B = A.T / (1.01 * la.norm(A, 2) ** 2)
    B_ = tf.Variable(B, dtype=tf.float32, name='B_0')
    By_ = tf.matmul(B_, prob.y_)
    S_ = tf.Variable(np.identity(N) - np.matmul(B, A), dtype=tf.float32, name='S_0')
    # pdb.set_trace()
    layers.append(('Linear', By_, None))

    initial_lambda = np.array(initial_lambda).astype(np.float32)
    # if getattr(prob,'iid',True) == False:
    #    # create a parameter for each coordinate in x
    #    initial_lambda = initial_lambda*np.ones( (N,1),dtype=np.float32 )
    lam0_ = tf.Variable(initial_lambda, name='lam_0')
    xhat_ = blocksoft(By_, lam0_)
    tk_ = tf.Variable((1 + np.sqrt(1 + 4 * 1 ** 2)) * 2 ** (-1), name='t_0')
    z_ = xhat_
    layers.append(('LBISTA T=1', xhat_, (lam0_,tk_)))
    # pdb.set_trace()
    for t in range(1, T):
        t_prev = tk_.read_value()
        t_prev_ = tk_
        xhat_prev_ = xhat_
        lam_ = tf.Variable(initial_lambda, name='lam_{0}'.format(t))
        xhat_ = blocksoft(tf.matmul(S_, z_) + By_, lam_)
        tk_ = tf.Variable((1 + tf.sqrt(1 + 4 * t_prev_ ** 2)) * 2 ** (-1), name='t_{0}'.format(t))
        z_ = xhat_ + (t_prev_ - 1) * (tk_) ** (-1) * (xhat_ - xhat_prev_)
        layers.append(('LBISTA T=' + str(t + 1), xhat_, (lam_,tk_)))

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers

def build_LBelastic_net(prob,T,initial_lambda=.1,untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    eta=block_soft_threshold_elastic_net
    layers = []
    A = prob.A
    M,N = A.shape
    B = A.T / (1.01 * la.norm(A,2)**2)
    B_ =  tf.Variable(B,dtype=tf.float32,name='B_0')
    By_ = tf.matmul(B_,prob.y_)
    S_ = tf.Variable( np.identity(N) - np.matmul(B,A),dtype=tf.float32,name='S_0')
    #pdb.set_trace()
    layers.append( ('Linear',By_,None) )
    
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    #if getattr(prob,'iid',True) == False:
    #    # create a parameter for each coordinate in x
    #    initial_lambda = initial_lambda*np.ones( (N,1),dtype=np.float32 )
    al0_ = tf.Variable( initial_lambda,name='al_0')
    lam0_ = tf.Variable( initial_lambda,name='lam_0')
    xhat_ = eta( By_, al0_, lam0_)
    layers.append( ('LBISTA T=1',xhat_, (al0_, lam0_) ) )
    #pdb.set_trace()
    for t in range(1,T):
        al_ = tf.Variable( initial_lambda,name='al_{0}'.format(t) )
        lam_ = tf.Variable( initial_lambda,name='lam_{0}'.format(t) )
        xhat_ = eta( tf.matmul(S_,xhat_) + By_, al_, lam_ )
        layers.append( ('LBISTA T='+str(t+1),xhat_,(al_, lam_)) )

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers
  
def build_UntiedLBelastic_net(prob,T,initial_lambda=.1,untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    eta=block_soft_threshold_elastic_net
    layers = []
    A = prob.A
    M,N = A.shape
    B = A.T / (1.01 * la.norm(A,2)**2)
    B_ =  tf.Variable(B,dtype=tf.float32,name='B_0')
    By_ = tf.matmul(B_,prob.y_)
    S_ = tf.Variable( np.identity(N) - np.matmul(B,A),dtype=tf.float32,name='S_0')
    #pdb.set_trace()
    #layers.append( ('Linear',By_,None) )
    
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    #if getattr(prob,'iid',True) == False:
    #    # create a parameter for each coordinate in x
    #    initial_lambda = initial_lambda*np.ones( (N,1),dtype=np.float32 )
    al0_ = tf.Variable( initial_lambda,name='al_0')
    lam0_ = tf.Variable( initial_lambda,name='lam_0')
    xhat_ = eta( By_, al0_, lam0_)
    layers.append( ('LBISTA T=1',xhat_, (al0_, lam0_, B_, S_) ) )
    #pdb.set_trace()
    for t in range(1,T):
        al_ = tf.Variable( initial_lambda,name='al_{0}'.format(t) )
        lam_ = tf.Variable( initial_lambda,name='lam_{0}'.format(t) )
        B_ =  tf.Variable(B,dtype=tf.float32,name='B_{0}'.format(t) )
        By_ = tf.matmul(B_,prob.y_)
        S_ = tf.Variable( np.identity(N) - np.matmul(B,A),dtype=tf.float32,name='S_{0}'.format(t) )
        xhat_ = eta( tf.matmul(S_,xhat_) + By_, al_, lam_ )
        layers.append( ('LBISTA T='+str(t+1),xhat_,(al_, lam_, B_, S_)) )

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers

def build_LBFastelastic_net(prob,T,initial_lambda=.1,untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    prob            - is a TFGenerator which contains problem parameters and def of how to generate training data
    initial_lambda  - could be some parameter of Block ISTA <- DELETE if unnecessary
    untied          - flag for tied or untied case
    Return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    eta=block_soft_threshold_elastic_net
    layers = []
    A = prob.A
    M,N = A.shape
    B = A.T / (1.01 * la.norm(A,2)**2)
    B_ =  tf.Variable(B,dtype=tf.float32,name='B_0')
    By_ = tf.matmul(B_,prob.y_)
    S_ = tf.Variable( np.identity(N) - np.matmul(B,A),dtype=tf.float32,name='S_0')
    #pdb.set_trace()
    layers.append( ('Linear',By_,None) )
    
    initial_lambda = np.array(initial_lambda).astype(np.float32)
    #if getattr(prob,'iid',True) == False:
    #    # create a parameter for each coordinate in x
    #    initial_lambda = initial_lambda*np.ones( (N,1),dtype=np.float32 )
    al0_ = tf.Variable( initial_lambda,name='al_0')
    lam0_ = tf.Variable( initial_lambda,name='lam_0')
    xhat_ = eta( By_, al0_, lam0_)
    tk = (1+np.sqrt(1+4*1**2))*2**(-1)
    z_ = xhat_
    layers.append( ('LBISTA T=1',xhat_, (al0_, lam0_) ) )
    #pdb.set_trace()
    for t in range(1,T):
        t_prev = tk
        xhat_prev_ = xhat_ 
        al_ = tf.Variable( initial_lambda,name='al_{0}'.format(t) )
        lam_ = tf.Variable( initial_lambda,name='lam_{0}'.format(t) )
        xhat_ = eta( tf.matmul(S_,z_) + By_, al_, lam_ )
        tk = (1+np.sqrt(1+4*t_prev**2))*2**(-1)
        z_ = xhat_ + (t_prev-1)*(tk)**(-1)*(xhat_-xhat_prev_)
        layers.append( ('LBISTA T='+str(t+1),xhat_,(al_, lam_)) )

    """
    # check other functions in this file (e.g., build_LISTA and build_LAMP4SSC) to implement LBISTA network
    # send me questions if needed
    """

    return layers


def build_LISTA(prob,T,initial_lambda=.1,untied=False):
    """
    Builds a LISTA network to infer x from prob.y_ = matmul(prob.A,x) + AWGN
    return a list of layer info (name,xhat_,newvars)
     name : description, e.g. 'LISTA T=1'
     xhat_ : that which approximates x_ at some point in the algorithm
     newvars : a tuple of layer-specific trainable variables
    """
    assert not untied,'TODO: untied'
    eta = simple_soft_threshold
    layers = []
    A = prob.A
    M,N = A.shape
    B = A.T / (1.01 * la.norm(A,2)**2)
    B_ =  tf.Variable(B,dtype=tf.float32,name='B_0')
    S_ = tf.Variable( np.identity(N) - np.matmul(B,A),dtype=tf.float32,name='S_0')
    By_ = tf.matmul( B_ , prob.y_ )
    layers.append( ('Linear',By_,None) )

    initial_lambda = np.array(initial_lambda).astype(np.float32)
    if getattr(prob,'iid',True) == False:
        # create a parameter for each coordinate in x
        initial_lambda = initial_lambda*np.ones( (N,1),dtype=np.float32 )
    lam0_ = tf.Variable( initial_lambda,name='lam_0')
    xhat_ = eta( By_, lam0_)
    #pdb.set_trace()
    layers.append( ('LISTA T=1',xhat_, (lam0_,) ) )
    for t in range(1,T):
        lam_ = tf.Variable( initial_lambda,name='lam_{0}'.format(t) )
        xhat_ = eta( tf.matmul(S_,xhat_) + By_, lam_ )
        layers.append( ('LISTA T='+str(t+1),xhat_,(lam_,)) )
    return layers

# Training

In [ ]:
def load_trainable_vars(sess, filename):
    """load a .npz archive and assign the value of each loaded
    ndarray to the trainable variable whose name matches the
    archive key.  Any elements in the archive that do not have
    a corresponding trainable variable will be returned in a dict.
    """
    other = {}
    try:
        tv = dict([(str(v.name), v) for v in tf.trainable_variables()])
        for k, d in np.load(filename).items():
            if k in tv:
                print('restoring ' + k + ' is:' + str(d))
                sess.run(tf.assign(tv[k], d))
            else:
                other[k] = d
    except IOError:
        pass
    return other


def save_trainable_vars(sess, filename, **kwargs):
    """save a .npz archive in `filename`  with
    the current value of each variable in tf.trainable_variables()
    plus any keyword numpy arrays.
    """
    save = {}
    for v in tf.trainable_variables():
        save[str(v.name)] = sess.run(v)
    save.update(kwargs)
    np.savez(filename, **save)


def setup_training(layer_info, prob, trinit=1e-3, refinements=(.5, .1, .01), final_refine=None):
    # with tf.device('/device:GPU:0'):
    """ Given a list of layer info (name,xhat_,newvars),
    create an output list of training operations (name,xhat_,loss_,nmse_,trainop_ ).
    Each layer_info element will be split into one or more output training operations
    based on the presence of newvars and len(refinements)
    """
    losses_ = []
    nmse_ = []
    trainers_ = []
    assert np.array(refinements).min() > 0, 'all refinements must be in (0,1]'
    assert np.array(refinements).max() <= 1, 'all refinements must be in (0,1]'

    maskX_ = getattr(prob, 'maskX_', 1)
    if maskX_ != 1:
        print('masking out inconsequential parts of signal x for nmse reporting')

    nmse_denom_ = tf.nn.l2_loss(prob.x_ * maskX_)

    tr_ = tf.Variable(trinit, name='tr', trainable=False)
    training_stages = []
    Wmat = sio.loadmat('WblockLag.mat')
    W = Wmat.get('W')
    t = 0
    mut_block = bst.mutual_block_coherence(W, prob.A, 32, 16)
    s = 2.0
    d = 8.0
    upper_bound = 2/(d*(2*mut_block*s-mut_block)+1)
    t = 0
    for name, xhat_, var_list in layer_info:
        gam_constraints = 0
        # pdb.set_trace()
        for i in range(0, len(tf.trainable_variables())):
            if tf.trainable_variables()[i].name.startswith('gam') and i==t:
                gam_constraints = gam_constraints + 100*tf.maximum(tf.trainable_variables()[i], upper_bound)
        t = t+1
        loss_ = tf.nn.l2_loss(xhat_ - prob.x_) + gam_constraints
        nmse_ = tf.nn.l2_loss((xhat_ - prob.x_) * maskX_) / nmse_denom_
        "sigma2_ = tf.reduce_mean(rvar_)"
        "sigma2_empirical_ = tf.reduce_mean((rhat_ - prob.x_)**2)"

        se_ = 2 * tf.nn.l2_loss(xhat_ - prob.x_)  # to get MSE, divide by / (L * N)

        #if name == 'LBISTA T=6':
        if var_list is not None:
            train_ = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list)
            training_stages.append((name, xhat_, loss_, nmse_, se_, train_, var_list))
        # for fm in refinements:
        #      train2_ = tf.train.AdamOptimizer(tr_ * fm).minimize(loss_)
        #      training_stages.append((name + ' trainrate=' + str(fm), xhat_, loss_, nmse_, se_, train2_, ()))
    if final_refine:
        train2_ = tf.train.AdamOptimizer(tr_ * final_refine).minimize(loss_)
        training_stages.append((name + ' final refine ' + str(final_refine), xhat_, loss_, nmse_, se_, train2_, ()))

    return training_stages



def setup_training_loss2(layer_info, prob, trinit=1e-3, refinements=(.5, .1, .01), final_refine=None):
    # with tf.device('/device:GPU:0'):
    """ Given a list of layer info (name,xhat_,newvars),
    create an output list of training operations (name,xhat_,loss_,nmse_,trainop_ ).
    Each layer_info element will be split into one or more output training operations
    based on the presence of newvars and len(refinements)
    """
    losses_ = []
    nmse_ = []
    trainers_ = []
    assert np.array(refinements).min() > 0, 'all refinements must be in (0,1]'
    assert np.array(refinements).max() <= 1, 'all refinements must be in (0,1]'

    maskX_ = getattr(prob, 'maskX_', 1)
    if maskX_ != 1:
        print('masking out inconsequential parts of signal x for nmse reporting')

    nmse_denom_ = tf.nn.l2_loss(prob.x_ * maskX_)

    tr_ = tf.Variable(trinit, name='tr', trainable=False)
    training_stages = []
    Wmat = sio.loadmat('WLag.mat')
    W = Wmat.get('W')
    t = 0
    for name, xhat_, var_list in layer_info:
        constraints = 0
        B = prob.B
        L = prob.L
        shape = K.shape(prob.x_)
        pool_shape1 = tf.stack([L, B, shape[1]])
        x_reshaped = K.reshape(prob.x_, pool_shape1)
        x_hat_reshaped = K.reshape(xhat_, pool_shape1)
        r_ = tf.sqrt(tf.reduce_sum(((x_reshaped-x_hat_reshaped) ** 2), 1))
        upper_bound = tf.reduce_max(r_)
        gam = 0
        lam = 0
        for i in range(0, len(var_list)):
            if var_list[i].name.startswith('gamma_'+str(t)):
                gam = var_list[i]
            if var_list[i].name.startswith('lam_'+str(t)):
                lam = var_list[i]

        # pdb.set_trace()
        constraints = constraints + 10*tf.maximum(lam/gam, upper_bound) - 10*tf.minimum(gam/1, 0)
        loss_ = tf.nn.l2_loss(xhat_ - prob.x_) + constraints
        nmse_ = tf.nn.l2_loss((xhat_ - prob.x_) * maskX_) / nmse_denom_
        t = t+1
        "sigma2_ = tf.reduce_mean(rvar_)"
        "sigma2_empirical_ = tf.reduce_mean((rhat_ - prob.x_)**2)"
        #pdb.set_trace()
        se_ = 2 * tf.nn.l2_loss(xhat_ - prob.x_)  # to get MSE, divide by / (L * N)

        #if name == 'LBISTA T=6':
        if var_list is not None:
            train_ = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list)
            training_stages.append((name, xhat_, loss_, nmse_, se_, train_, var_list))
        # for fm in refinements:
        #      train2_ = tf.train.AdamOptimizer(tr_ * fm).minimize(loss_)
        #      training_stages.append((name + ' trainrate=' + str(fm), xhat_, loss_, nmse_, se_, train2_, ()))
    if final_refine:
        train2_ = tf.train.AdamOptimizer(tr_ * final_refine).minimize(loss_)
        training_stages.append((name + ' final refine ' + str(final_refine), xhat_, loss_, nmse_, se_, train2_, ()))

    return training_stages


def setup_training_loss3(layer_info, prob, trinit=1e-3, refinements=(.5, .1, .01), final_refine=None):
    # with tf.device('/device:GPU:0'):
    """ Given a list of layer info (name,xhat_,newvars),
    create an output list of training operations (name,xhat_,loss_,nmse_,trainop_ ).
    Each layer_info element will be split into one or more output training operations
    based on the presence of newvars and len(refinements)
    """
    losses_ = []
    nmse_ = []
    trainers_ = []
    assert np.array(refinements).min() > 0, 'all refinements must be in (0,1]'
    assert np.array(refinements).max() <= 1, 'all refinements must be in (0,1]'

    maskX_ = getattr(prob, 'maskX_', 1)
    if maskX_ != 1:
        print('masking out inconsequential parts of signal x for nmse reporting')

    nmse_denom_ = tf.nn.l2_loss(prob.x_ * maskX_)

    tr_ = tf.Variable(trinit, name='tr', trainable=False)
    training_stages = []
    Wmat = sio.loadmat('WLag.mat')
    W = Wmat.get('W')
    t = 0
    for name, xhat_, var_list in layer_info:
        constraints = 0
        mut_block = bst.mutual_block_coherence(W, prob.A, prob.L, prob.B)
        B = prob.B
        L = prob.L
        C_w = bst.C_w(W, prob.L, prob.B)
        sigma = np.linalg.norm(prob.noise, 2)
        shape = K.shape(prob.x_)
        pool_shape1 = tf.stack([L, B, shape[1]])
        x_reshaped = K.reshape(prob.x_, pool_shape1)
        x_hat_reshaped = K.reshape(xhat_, pool_shape1)
        r_ = tf.sqrt(tf.reduce_sum(((x_reshaped-x_hat_reshaped) ** 2), 1))
        lower_bound = B*mut_block*tf.reduce_max(r_)+C_w*sigma
        gam = 0
        lam = 0
        for i in range(0, len(var_list)):
            if var_list[i].name.startswith('gamma_'+str(t)):
                gam = var_list[i]
            if var_list[i].name.startswith('lam_'+str(t)):
                lam = var_list[i]

        # pdb.set_trace()
        constraints = constraints - 2*tf.minimum(lam/1, (gam/1)*(lower_bound)) - 2*tf.minimum(gam/1, 0)
        loss_ = tf.nn.l2_loss(xhat_ - prob.x_) + constraints
        nmse_ = tf.nn.l2_loss((xhat_ - prob.x_) * maskX_) / nmse_denom_
        t = t+1
        "sigma2_ = tf.reduce_mean(rvar_)"
        "sigma2_empirical_ = tf.reduce_mean((rhat_ - prob.x_)**2)"
        #pdb.set_trace()
        se_ = 2 * tf.nn.l2_loss(xhat_ - prob.x_)  # to get MSE, divide by / (L * N)

        #if name == 'LBISTA T=6':
        if var_list is not None:
            train_ = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list)
            training_stages.append((name, xhat_, loss_, nmse_, se_, train_, var_list))
        # for fm in refinements:
        #      train2_ = tf.train.AdamOptimizer(tr_ * fm).minimize(loss_)
        #      training_stages.append((name + ' trainrate=' + str(fm), xhat_, loss_, nmse_, se_, train2_, ()))
    if final_refine:
        train2_ = tf.train.AdamOptimizer(tr_ * final_refine).minimize(loss_)
        training_stages.append((name + ' final refine ' + str(final_refine), xhat_, loss_, nmse_, se_, train2_, ()))

    return training_stages


def setup_training_loss4(layer_info, prob, trinit=1e-3, refinements=(.5, .1, .01), final_refine=None):
    # with tf.device('/device:GPU:0'):
    """ Given a list of layer info (name,xhat_,newvars),
    create an output list of training operations (name,xhat_,loss_,nmse_,trainop_ ).
    Each layer_info element will be split into one or more output training operations
    based on the presence of newvars and len(refinements)
    """
    losses_ = []
    nmse_ = []
    trainers_ = []
    assert np.array(refinements).min() > 0, 'all refinements must be in (0,1]'
    assert np.array(refinements).max() <= 1, 'all refinements must be in (0,1]'

    maskX_ = getattr(prob, 'maskX_', 1)
    if maskX_ != 1:
        print('masking out inconsequential parts of signal x for nmse reporting')

    nmse_denom_ = tf.nn.l2_loss(prob.x_ * maskX_)

    tr_ = tf.Variable(trinit, name='tr', trainable=False)
    training_stages = []
    Wmat = sio.loadmat('WLag.mat')
    W = Wmat.get('W')
    t = 0
    for name, xhat_, var_list in layer_info:
        constraints = 0
        mut_block = bst.mutual_block_coherence(W, prob.A, prob.L, prob.B)
        B = prob.B
        L = prob.L
        C_w = bst.C_w(W, prob.L, prob.B)
        sigma = np.linalg.norm(prob.noise, 2)
        shape = K.shape(prob.x_)
        pool_shape1 = tf.stack([L, B, shape[1]])
        x_reshaped = K.reshape(prob.x_, pool_shape1)
        x_hat_reshaped = K.reshape(xhat_, pool_shape1)
        r_ = tf.sqrt(tf.reduce_sum(((x_reshaped-x_hat_reshaped) ** 2), 1))
        lower_bound = B*mut_block*tf.reduce_max(r_)+C_w*sigma
        gam = 0
        lam = 0
        for i in range(0, len(var_list)):
            if var_list[i].name.startswith('gamma_'+str(t)):
                gam = var_list[i]
            if var_list[i].name.startswith('lam_'+str(t)):
                lam = var_list[i]

        # pdb.set_trace()
        constraints = constraints - 2*tf.minimum(lam/1, (gam/1)*(lower_bound)) - 2*tf.minimum(gam/1, 0)
        loss_ = tf.nn.l2_loss(xhat_ - prob.x_) + constraints
        nmse_ = tf.nn.l2_loss((xhat_ - prob.x_) * maskX_) / nmse_denom_
        t = t+1
        "sigma2_ = tf.reduce_mean(rvar_)"
        "sigma2_empirical_ = tf.reduce_mean((rhat_ - prob.x_)**2)"
        #pdb.set_trace()
        se_ = 2 * tf.nn.l2_loss(xhat_ - prob.x_)  # to get MSE, divide by / (L * N)

        #if name == 'LBISTA T=6':
        if var_list is not None: #IDEA train at first only gamma, after this learn lambdas...
            train_gam = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list[1])
            training_stages.append((name+'only Gamma', xhat_, loss_, nmse_, se_, train_gam, [var_list[1]]))
            train_lam = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list[0])
            training_stages.append((name+'only Lambda', xhat_, loss_, nmse_, se_, train_lam, [var_list[0]]))

        # for fm in refinements:
        #      train2_ = tf.train.AdamOptimizer(tr_ * fm).minimize(loss_)
        #      training_stages.append((name + ' trainrate=' + str(fm), xhat_, loss_, nmse_, se_, train2_, ()))
    if final_refine:
        train2_ = tf.train.AdamOptimizer(tr_ * final_refine).minimize(loss_)
        training_stages.append((name + ' final refine ' + str(final_refine), xhat_, loss_, nmse_, se_, train2_, ()))

    return training_stages


def setup_training_loss_max(layer_info, prob, trinit=1e-3, refinements=(.5, .1, .01), omega=5, final_refine=None):
    # with tf.device('/device:GPU:0'):
    """ Given a list of layer info (name,xhat_,newvars),
    create an output list of training operations (name,xhat_,loss_,nmse_,trainop_ ).
    Each layer_info element will be split into one or more output training operations
    based on the presence of newvars and len(refinements)
    """
    losses_ = []
    nmse_ = []
    trainers_ = []
    assert np.array(refinements).min() > 0, 'all refinements must be in (0,1]'
    assert np.array(refinements).max() <= 1, 'all refinements must be in (0,1]'

    maskX_ = getattr(prob, 'maskX_', 1)
    if maskX_ != 1:
        print('masking out inconsequential parts of signal x for nmse reporting')

    nmse_denom_ = tf.nn.l2_loss(prob.x_ * maskX_)

    tr_ = tf.Variable(trinit, name='tr', trainable=False)
    training_stages = []
    Wmat = sio.loadmat('WLag.mat')
    W = Wmat.get('W')
    sigma = np.ceil(np.mean(np.linalg.norm(prob.noise, 2, axis=0)))
    t = 0
    for name, xhat_, var_list in layer_info:
        constraints = 0
        # pdb.set_trace()
        mut_block = bst.mutual_block_coherence(W, prob.A, prob.m, prob.L, prob.B)
        C_w = bst.C_w(W, prob.L, prob.B)
        shape = K.shape(prob.x_)
        pool_shape1 = tf.stack([prob.L, prob.B, shape[1]])
        x_reshaped = K.reshape(prob.x_, pool_shape1)
        x_hat_reshaped = K.reshape(xhat_, pool_shape1)
        r_ = tf.sqrt(tf.reduce_sum(((x_reshaped-x_hat_reshaped) ** 2), 1))
        r_ = tf.reduce_sum(r_,0)
        lower_bound = prob.B*mut_block*tf.reduce_max(r_)+C_w*sigma
        # gam = 0
        # lam = 0
        for i in range(0, len(var_list)):
            if var_list[i].name.startswith('gamma_'+str(t)):
                gam = var_list[i]
            if var_list[i].name.startswith('lam_'+str(t)):
                lam = var_list[i]

        # pdb.set_trace()
        constraints = constraints
        loss_ = tf.nn.l2_loss(xhat_ - prob.x_)
        #loss_2 = tf.nn.l2_loss(xhat_ - prob.x_) - omega*tf.minimum(lam/1, (tf.math.abs(gam)/1)*(lower_bound))
        nmse_ = tf.nn.l2_loss((xhat_ - prob.x_) * maskX_) / nmse_denom_
        t = t+1
        "sigma2_ = tf.reduce_mean(rvar_)"
        "sigma2_empirical_ = tf.reduce_mean((rhat_ - prob.x_)**2)"
        #pdb.set_trace()
        se_ = 2 * tf.nn.l2_loss(xhat_ - prob.x_)  # to get MSE, divide by / (L * N)

        #if name == 'LBISTA T=6':
        if var_list is not None: #IDEA train at first only gamma, after this learn lambdas...
            train_gam = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list[1])
            training_stages.append((name+' only Gamma', xhat_, loss_, nmse_, se_, train_gam, [var_list[1]]))
            loss_2 = loss_ - omega*tf.minimum(var_list[0]/1 , (tf.math.abs(var_list[1])/1)*(lower_bound))
            train_lam = tf.train.AdamOptimizer(tr_).minimize(loss_2, var_list=var_list[0])
            training_stages.append((name+' only Lambda', xhat_, loss_, nmse_, se_, train_lam, [var_list[0]])) #welches loss mur nach xhat stehen?

        # for fm in refinements:
        #      train2_ = tf.train.AdamOptimizer(tr_ * fm).minimize(loss_)
        #      training_stages.append((name + ' trainrate=' + str(fm), xhat_, loss_, nmse_, se_, train2_, ()))
    if final_refine:
        train2_ = tf.train.AdamOptimizer(tr_ * final_refine).minimize(loss_)
        training_stages.append((name + ' final refine ' + str(final_refine), xhat_, loss_, nmse_, se_, train2_, ()))

    return training_stages

def setup_training_loss_mean(layer_info, prob, trinit=1e-3, refinements=(.5, .1, .01), omega = 5, final_refine=None):
    # with tf.device('/device:GPU:0'):
    """ Given a list of layer info (name,xhat_,newvars),
    create an output list of training operations (name,xhat_,loss_,nmse_,trainop_ ).
    Each layer_info element will be split into one or more output training operations
    based on the presence of newvars and len(refinements)
    """
    losses_ = []
    nmse_ = []
    trainers_ = []
    assert np.array(refinements).min() > 0, 'all refinements must be in (0,1]'
    assert np.array(refinements).max() <= 1, 'all refinements must be in (0,1]'

    maskX_ = getattr(prob, 'maskX_', 1)
    if maskX_ != 1:
        print('masking out inconsequential parts of signal x for nmse reporting')

    nmse_denom_ = tf.nn.l2_loss(prob.x_ * maskX_)

    tr_ = tf.Variable(trinit, name='tr', trainable=False)
    training_stages = []
    Wmat = sio.loadmat('WLag.mat')
    W = Wmat.get('W')
    sigma = np.ceil(np.mean(np.linalg.norm(prob.noise, 2, axis=0)))
    t = 0
    for name, xhat_, var_list in layer_info:
        constraints = 0
        # pdb.set_trace()
        mut_block = bst.mutual_block_coherence(W, prob.A, prob.m, prob.L, prob.B)
        C_w = bst.C_w(W, prob.L, prob.B)
        shape = K.shape(prob.x_)
        pool_shape1 = tf.stack([prob.L, prob.B, shape[1]])
        x_reshaped = K.reshape(prob.x_, pool_shape1)
        x_hat_reshaped = K.reshape(xhat_, pool_shape1)
        r_ = tf.sqrt(tf.reduce_sum(((x_reshaped-x_hat_reshaped) ** 2), 1))
        r_ = tf.reduce_sum(r_,0)
        lower_bound = prob.B*mut_block*tf.reduce_mean(r_)+C_w*sigma
        # gam = 0
        # lam = 0
        for i in range(0, len(var_list)):
            if var_list[i].name.startswith('gamma_'+str(t)):
                gam = var_list[i]
            if var_list[i].name.startswith('lam_'+str(t)):
                lam = var_list[i]

        # pdb.set_trace()
        constraints = constraints
        loss_ = tf.nn.l2_loss(xhat_ - prob.x_)
        #loss_2 = tf.nn.l2_loss(xhat_ - prob.x_) - omega*tf.minimum(lam/1, (tf.math.abs(gam)/1)*(lower_bound))
        nmse_ = tf.nn.l2_loss((xhat_ - prob.x_) * maskX_) / nmse_denom_
        t = t+1
        "sigma2_ = tf.reduce_mean(rvar_)"
        "sigma2_empirical_ = tf.reduce_mean((rhat_ - prob.x_)**2)"
        #pdb.set_trace()
        se_ = 2 * tf.nn.l2_loss(xhat_ - prob.x_)  # to get MSE, divide by / (L * N)

        #if name == 'LBISTA T=6':
        if var_list is not None: #IDEA train at first only gamma, after this learn lambdas...
            train_gam = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list[1])
            training_stages.append((name+' only Gamma', xhat_, loss_, nmse_, se_, train_gam, [var_list[1]]))
            loss_2 = loss_ - omega*tf.minimum([var_list[0]/1 , (tf.math.abs(var_list[1])/1)*(lower_bound)])
            train_lam = tf.train.AdamOptimizer(tr_).minimize(loss_2, var_list=var_list[0])
            training_stages.append((name+' only Lambda', xhat_, loss_, nmse_, se_, train_lam, [var_list[0]])) #welches loss mur nach xhat stehen?

        # for fm in refinements:
        #      train2_ = tf.train.AdamOptimizer(tr_ * fm).minimize(loss_)
        #      training_stages.append((name + ' trainrate=' + str(fm), xhat_, loss_, nmse_, se_, train2_, ()))
    if final_refine:
        train2_ = tf.train.AdamOptimizer(tr_ * final_refine).minimize(loss_)
        training_stages.append((name + ' final refine ' + str(final_refine), xhat_, loss_, nmse_, se_, train2_, ()))

    return training_stages

def setup_training_loss_mean_LSE(layer_info, prob, trinit=1e-3, refinements=(.5, .1, .01),omega=5, final_refine=None):
    # with tf.device('/device:GPU:0'):
    """ Given a list of layer info (name,xhat_,newvars),
    create an output list of training operations (name,xhat_,loss_,nmse_,trainop_ ).
    Each layer_info element will be split into one or more output training operations
    based on the presence of newvars and len(refinements)
    """
    losses_ = []
    nmse_ = []
    trainers_ = []
    assert np.array(refinements).min() > 0, 'all refinements must be in (0,1]'
    assert np.array(refinements).max() <= 1, 'all refinements must be in (0,1]'

    maskX_ = getattr(prob, 'maskX_', 1)
    if maskX_ != 1:
        print('masking out inconsequential parts of signal x for nmse reporting')

    nmse_denom_ = tf.nn.l2_loss(prob.x_ * maskX_)

    tr_ = tf.Variable(trinit, name='tr', trainable=False)
    training_stages = []
    Wmat = sio.loadmat('W_cvx.mat')
    W = Wmat.get('W_cvx')
    sigma = np.ceil(np.mean(np.linalg.norm(prob.noise, 2, axis=0)))
    t = 0
    for name, xhat_, var_list in layer_info:
        constraints = 0
        # pdb.set_trace()
        mut_block = bst.mutual_block_coherence(W, prob.A, prob.m, prob.L, prob.B)
        C_w = bst.C_w(W, prob.L, prob.B)
        shape = K.shape(prob.x_)
        pool_shape1 = tf.stack([prob.L, prob.B, shape[1]])
        x_reshaped = K.reshape(prob.x_, pool_shape1)
        x_hat_reshaped = K.reshape(xhat_, pool_shape1)
        r_ = tf.sqrt(tf.reduce_sum(((x_reshaped-x_hat_reshaped) ** 2), 1))
        r_ = tf.reduce_sum(r_,0)
        lower_bound = prob.B*mut_block*tf.reduce_mean(r_)+C_w*sigma
        # pdb.set_trace()
        constraints = constraints
        loss_ = tf.nn.l2_loss(xhat_ - prob.x_)
        #loss_2 = tf.nn.l2_loss(xhat_ - prob.x_) + gam*tf.math.reduce_logsumexp([ - lam/1 , - (tf.math.abs(gam)/1)*(lower_bound)])
        nmse_ = tf.nn.l2_loss((xhat_ - prob.x_) * maskX_) / nmse_denom_
        t = t+1
        "sigma2_ = tf.reduce_mean(rvar_)"
        "sigma2_empirical_ = tf.reduce_mean((rhat_ - prob.x_)**2)"
        #pdb.set_trace()
        se_ = 2 * tf.nn.l2_loss(xhat_ - prob.x_)  # to get MSE, divide by / (L * N)

        #if name == 'LBISTA T=6':
        if var_list is not None: #IDEA train at first only gamma, after this learn lambdas...
            train_gam = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list[1])
            training_stages.append((name+' only Gamma', xhat_, loss_, nmse_, se_, train_gam, [var_list[1]]))
            loss_2 = loss_ + omega*tf.math.reduce_logsumexp([ - var_list[0]/1 , - (tf.math.abs(var_list[1])/1)*(lower_bound)])
            train_lam = tf.train.AdamOptimizer(tr_).minimize(loss_2, var_list=var_list[0])
            training_stages.append((name+' only Lambda', xhat_, loss_, nmse_, se_, train_lam, [var_list[0]])) #welches loss mur nach xhat stehen?

        # for fm in refinements:
        #      train2_ = tf.train.AdamOptimizer(tr_ * fm).minimize(loss_)
        #      training_stages.append((name + ' trainrate=' + str(fm), xhat_, loss_, nmse_, se_, train2_, ()))
    if final_refine:
        train2_ = tf.train.AdamOptimizer(tr_ * final_refine).minimize(loss_)
        training_stages.append((name + ' final refine ' + str(final_refine), xhat_, loss_, nmse_, se_, train2_, ()))

    return training_stages


def setup_training_loss_max_LSE(layer_info, prob, trinit=1e-3, refinements=(.5, .1, .01),omega=5, final_refine=None):
    # with tf.device('/device:GPU:0'):
    """ Given a list of layer info (name,xhat_,newvars),
    create an output list of training operations (name,xhat_,loss_,nmse_,trainop_ ).
    Each layer_info element will be split into one or more output training operations
    based on the presence of newvars and len(refinements)
    """
    losses_ = []
    nmse_ = []
    trainers_ = []
    assert np.array(refinements).min() > 0, 'all refinements must be in (0,1]'

    assert np.array(refinements).max() <= 1, 'all refinements must be in (0,1]'

    maskX_ = getattr(prob, 'maskX_', 1)
    if maskX_ != 1:
        print('masking out inconsequential parts of signal x for nmse reporting')

    nmse_denom_ = tf.nn.l2_loss(prob.x_ * maskX_)

    tr_ = tf.Variable(trinit, name='tr', trainable=False)
    training_stages = []
    Wmat = sio.loadmat('small_normalized_W_cvx.mat')
    W = Wmat.get('W_cvx')
    sigma = np.ceil(np.mean(np.linalg.norm(prob.noise, 2, axis=0)))
    t = 0
    for name, xhat_, var_list in layer_info:
        constraints = 0
        # pdb.set_trace()
        mut_block = bst.mutual_block_coherence(W, prob.A, prob.m, prob.L, prob.B)
        C_w = bst.C_w(W, prob.L, prob.B)
        shape = K.shape(prob.x_)
        pool_shape1 = tf.stack([prob.L, prob.B, shape[1]])
        x_reshaped = K.reshape(prob.x_, pool_shape1)
        x_hat_reshaped = K.reshape(xhat_, pool_shape1)
        r_ = tf.sqrt(tf.reduce_sum(((x_reshaped-x_hat_reshaped) ** 2), 1))
        r_ = tf.reduce_sum(r_,0)
        lower_bound = prob.B*mut_block*tf.reduce_max(r_)+C_w*sigma
        # pdb.set_trace()
        constraints = constraints
        loss_ = tf.nn.l2_loss(xhat_ - prob.x_)
        #loss_2 = tf.nn.l2_loss(xhat_ - prob.x_) + gam*tf.math.reduce_logsumexp([ - lam/1 , - (tf.math.abs(gam)/1)*(lower_bound)])
        nmse_ = tf.nn.l2_loss((xhat_ - prob.x_) * maskX_) / nmse_denom_
        t = t+1
        "sigma2_ = tf.reduce_mean(rvar_)"
        "sigma2_empirical_ = tf.reduce_mean((rhat_ - prob.x_)**2)"
        #pdb.set_trace()
        se_ = 2 * tf.nn.l2_loss(xhat_ - prob.x_)  # to get MSE, divide by / (L * N)

        #if name == 'LBISTA T=6':
        if var_list is not None: #IDEA train at first only gamma, after this learn lambdas...
            train_gam = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list[1])
            training_stages.append((name+' only Gamma', xhat_, loss_, nmse_, se_, train_gam, [var_list[1]]))
            loss_2 = loss_ + omega*tf.math.reduce_logsumexp([ - var_list[0]/1 , - (tf.math.abs(var_list[1])/1)*(lower_bound)])
            train_lam = tf.train.AdamOptimizer(tr_).minimize(loss_2, var_list=var_list[0])
            training_stages.append((name+' only Lambda', xhat_, loss_2, nmse_, se_, train_lam, [var_list[0]])) #welches loss mur nach xhat stehen?

        # for fm in refinements:
        #      train2_ = tf.train.AdamOptimizer(tr_ * fm).minimize(loss_)
        #      training_stages.append((name + ' trainrate=' + str(fm), xhat_, loss_, nmse_, se_, train2_, ()))
    if final_refine:
        train2_ = tf.train.AdamOptimizer(tr_ * final_refine).minimize(loss_)
        training_stages.append((name + ' final refine ' + str(final_refine), xhat_, loss_, nmse_, se_, train2_, ()))

    return training_stages

def setup_training_loss_mean_LSE_learn_omega(layer_info, prob, trinit=1e-3, refinements=(.5, .1, .01), final_refine=None):
    # with tf.device('/device:GPU:0'):
    """ Given a list of layer info (name,xhat_,newvars),
    create an output list of training operations (name,xhat_,loss_,nmse_,trainop_ ).
    Each layer_info element will be split into one or more output training operations
    based on the presence of newvars and len(refinements)
    """
    losses_ = []
    nmse_ = []
    trainers_ = []
    assert np.array(refinements).min() > 0, 'all refinements must be in (0,1]'
    assert np.array(refinements).max() <= 1, 'all refinements must be in (0,1]'

    maskX_ = getattr(prob, 'maskX_', 1)
    if maskX_ != 1:
        print('masking out inconsequential parts of signal x for nmse reporting')

    nmse_denom_ = tf.nn.l2_loss(prob.x_ * maskX_)

    tr_ = tf.Variable(trinit, name='tr', trainable=False)
    training_stages = []
    Wmat = sio.loadmat('small_normalized_W_cvx.mat')
    W = Wmat.get('W_cvx')
    sigma = np.ceil(np.mean(np.linalg.norm(prob.noise, 2, axis=0)))
    t = 0
    for name, xhat_, var_list in layer_info:
        constraints = 0
        # pdb.set_trace()
        mut_block = bst.mutual_block_coherence(W, prob.A, prob.m, prob.L, prob.B)
        C_w = bst.C_w(W, prob.L, prob.B)
        shape = K.shape(prob.x_)
        pool_shape1 = tf.stack([prob.L, prob.B, shape[1]])
        x_reshaped = K.reshape(prob.x_, pool_shape1)
        x_hat_reshaped = K.reshape(xhat_, pool_shape1)
        r_ = tf.sqrt(tf.reduce_sum(((x_reshaped-x_hat_reshaped) ** 2), 1))
        r_ = tf.reduce_sum(r_,0)
        lower_bound = prob.B*mut_block*tf.reduce_mean(r_)+C_w*sigma
        # gam = 0
        # lam = 0
        for i in range(0, len(var_list)):
            if var_list[i].name.startswith('gamma_'+str(t)):
                gam = var_list[i]
            if var_list[i].name.startswith('lam_'+str(t)):
                lam = var_list[i]

        # pdb.set_trace()
        constraints = constraints
        omega_ = tf.Variable(5, dtype=tf.float32, name='omega_')
        loss_ = tf.nn.l2_loss(xhat_ - prob.x_)
        loss_2 = tf.nn.l2_loss(xhat_ - prob.x_) + omega_*tf.math.reduce_logsumexp([ - lam/1 , - (tf.math.abs(gam)/1)*(lower_bound)])
        nmse_ = tf.nn.l2_loss((xhat_ - prob.x_) * maskX_) / nmse_denom_
        t = t+1
        "sigma2_ = tf.reduce_mean(rvar_)"
        "sigma2_empirical_ = tf.reduce_mean((rhat_ - prob.x_)**2)"
        #pdb.set_trace()
        se_ = 2 * tf.nn.l2_loss(xhat_ - prob.x_)  # to get MSE, divide by / (L * N)

        #if name == 'LBISTA T=6':
        if var_list is not None: #IDEA train at first only gamma, after this learn lambdas...
            train_gam = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list[1])
            training_stages.append((name+' only Gamma', xhat_, loss_, nmse_, se_, train_gam, [var_list[1]]))
            train_lam = tf.train.AdamOptimizer(tr_).minimize(loss_2, var_list=[var_list[0],omega_])
            training_stages.append((name+' only Lambda', xhat_, loss_, nmse_, se_, train_lam, [var_list[0],omega_])) #welches loss mur nach xhat stehen?

        # for fm in refinements:
        #      train2_ = tf.train.AdamOptimizer(tr_ * fm).minimize(loss_)
        #      training_stages.append((name + ' trainrate=' + str(fm), xhat_, loss_, nmse_, se_, train2_, ()))
    if final_refine:
        train2_ = tf.train.AdamOptimizer(tr_ * final_refine).minimize(loss_)
        training_stages.append((name + ' final refine ' + str(final_refine), xhat_, loss_, nmse_, se_, train2_, ()))

    return training_stages



def do_training(training_stages, prob, savefile, ivl=10, maxit=100000, better_wait=5000):
    # with tf.device('/device:GPU:0'):
    """
    ivl:how often should we compute the nmse of the validation set?
    maxit: max number of training iterations
    better_wait:wait this many iterations for an nmse that is better than the prevoius best of the current training session
    """
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    print('norms xval:{xval:.7f} yval:{yval:.7f}'.format(xval=la.norm(prob.xval), yval=la.norm(prob.yval)))

    state = load_trainable_vars(sess, savefile)  # must load AFTER the initializer

    # must use this same Session to perform all training
    # if we start a new Session, things would replay and we'd be training with our validation set (no no)

    done = state.get('done', [])
    log = str(state.get('log', ''))

    for name, xhat_, loss_, nmse_, se_, train_, var_list in training_stages:
        start = time.time()
        if name in done:
            print('Already did ' + name + '. Skipping.')
            continue
        if len(var_list):
            describe_var_list = 'extending ' + ','.join([v.name for v in var_list])
        else:
            describe_var_list = 'fine tuning all ' + ','.join([v.name for v in tf.trainable_variables()])

        print(name + ' ' + describe_var_list)
        nmse_history = []

        for i in range(maxit + 1):

            if i % ivl == 0:
                nmse = sess.run(nmse_, feed_dict={prob.y_: prob.yval, prob.x_: prob.xval})
                if np.isnan(nmse):
                    #pdb.set_trace()
                    raise RuntimeError('nmse is NaN')
                nmse_history = np.append(nmse_history, nmse)
                nmse_dB = 10 * np.log10(nmse)
                nmsebest_dB = 10 * np.log10(nmse_history.min())
                sys.stdout.write(
                    '\ri={i:<6d} nmse={nmse:.6f} dB (best={best:.6f})'.format(i=i, nmse=nmse_dB, best=nmsebest_dB))
                sys.stdout.flush()
                if i % (100 * ivl) == 0:
                    print('')
                    age_of_best = len(nmse_history) - nmse_history.argmin() - 1  # how long ago was the best nmse?
                    if age_of_best * ivl > better_wait:
                        break  # if it has not improved on the best answer for quite some time, then move along
            y, x = prob(sess)
            sess.run(train_, feed_dict={prob.y_: y, prob.x_: x})  # hier fehler lam0=nan

        done = np.append(done, name)

        end = time.time()
        time_log = 'Took me {totaltime:.3f} minutes, or {time_per_interation:.1f} ms per iteration'.format(
            totaltime=(end - start) / 60, time_per_interation=(end - start) * 1000 / i)
        print(time_log)
        log = log + '\n{name} nmse={nmse:.6f} dB in {i} iterations'.format(name=name, nmse=nmse_dB, i=i)
        # pdb.set_trace()

        state['done'] = done
        state['log'] = log
        save_trainable_vars(sess, savefile, **state)
    return sess

# Main Part

In [ ]:
prob = block_gaussian_trial(m=50, L=15, B=5, MC=250, pnz=.1, SNR_dB=-10) # a Block-Gaussian x, noisily observed through a random matrix
T=16
layers,W = build_BALISTA_v5(prob,T=T,initial_lambda=.1,untied=False)
#layers = build_LBFISTA(prob,T=6,initial_lambda=.1,untied=False)
#layers = build_LBelastic_net(prob,T=6,initial_lambda=.1,untied=False)
#layers = build_LBFastelastic_net(prob,T=6,initial_lambda=.1,untied=False)
#layers = build_UntiedLBelastic_net(prob,T=6,initial_lambda=.1,untied=False)
start = time.time()
training_stages = setup_training_loss_max_LSE(layers,prob,trinit=1e-3,refinements=(.5,0.1,.01))
end = time.time()
print( 'Took me {totaltime:.3f} minutes for setup training'.format(totaltime = (end-start)/60))

In [ ]:
sess = do_training(training_stages,prob,'LBISTA_block_Gauss_giidT16Thermo6-JanWithGamConstraints-10.npz')

# Evaluating

In [ ]:
sparsemat = sio.loadmat('D_blocktestSNR' + str(-10) + '.mat')
y = sparsemat.get('y')
x = sparsemat.get('x')
MC = 250
t=0
l2norm=np.zeros(((T),MC))
nmse=np.zeros(((T),MC))
for name, xhat_, var_list in layers:
    if not name=='Linear':
        xhat = sess.run(xhat_, feed_dict={prob.y_: y, prob.x_: x})
        for i in range(0, x.shape[1]):
            nmse[t,i]=bst.nmse(xhat[:,i, np.newaxis], x[:,i, np.newaxis])
            l2norm[t, i] = bst.l21norm(xhat[:, i]- x[:, i], prob.L, prob.B)
        t+=1

In [ ]:
lam = np.zeros(T)
gam = np.zeros(T)
for i in range(0,T):
  lam[i]=sess.run(layers[i][2][0])
  gam[i]=sess.run(layers[i][2][1])

In [ ]:
plt.plot(lam/gam)

In [ ]:
plt.plot(lam)

In [ ]:
plt.plot(gam)

In [ ]:
plt.plot(10*np.log10(np.mean(np.ma.masked_invalid(nmse), axis=1)))

In [ ]:
plt.plot(xhat[:,99])
plt.plot(x[:,99])

In [ ]:
plt.plot(x[:,99])